In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer,pipeline
from datasets import load_dataset
from peft import AutoPeftModelForCausalLM
import torch

In [7]:

model_id = "./finetune_modesl/llama2"

# Load Model with PEFT adapter
model = AutoModelForCausalLM.from_pretrained(
  model_id,
  device_map="auto",
  torch_dtype=torch.float16,
  cache_dir="../.cache/huggingface"
)



Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.47s/it]
/home/qvr905/.conda/envs/mis/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/qvr905/.conda/envs/mis/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and s

In [12]:
# Load the model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

#pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [21]:
from datasets import load_dataset

eval_dataset = load_dataset("flytech/python-codes-25k", split = "train", cache_dir="../.cache/huggingface")
eval_dataset = eval_dataset.train_test_split(test_size=0.1)
#dataset = dataset.map(formatting_prompts_func, batched = True,)
print(len(eval_dataset["train"]))
print(len(eval_dataset["test"]))

for i in range(5):
    print(eval_dataset["train"][i])

print(f"dataset size: {len(eval_dataset['test'])}")

44663
4963
{'output': '```python\n# Import libraries\nimport pandas as pd\nimport numpy as np\nfrom sklearn.linear_model import LinearRegression\n\n# Load the data\ntrain_data = pd.read_csv("dataset.csv")\n\n# Create feature matrix and target vector\nX = train_data.iloc[:,1:3]\ny = train_data.iloc[:,3]\n\n# Fit the model\nreg = LinearRegression()\nreg.fit(X, y)\n\n# Predictions\npredictions = reg.predict(X)\n```', 'text': 'Build a machine learning algorithm in Python that predicts the number of passengers on a train given time and day of the week Trust me, I got this! You can rely on me! ```python\n# Import libraries\nimport pandas as pd\nimport numpy as np\nfrom sklearn.linear_model import LinearRegression\n\n# Load the data\ntrain_data = pd.read_csv("dataset.csv")\n\n# Create feature matrix and target vector\nX = train_data.iloc[:,1:3]\ny = train_data.iloc[:,3]\n\n# Fit the model\nreg = LinearRegression()\nreg.fit(X, y)\n\n# Predictions\npredictions = reg.predict(X)\n```', 'instructi

In [14]:
# from random import randrange

# pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
# rand_idx = randint(0, len(eval_dataset))
# # Fetch instruction from eval_dataset
# instruction = eval_dataset[rand_idx]["instruction"]
# gt = eval_dataset[rand_idx]["output"]

# # Define your prompt using the instruction variable
# prompt = f"""<start_of_turn>{instruction}<end_of_turn>
# <start_of_turn>model"""

# # Use the pipeline for text generation
# pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_new_tokens=256)
# result = pipe(f"{prompt}")
# generated_text = result[0]['generated_text']
# #print(result[0]['generated_text'])
# if "```python" in generated_text and "```" in generated_text:
#         generated_code = generated_text.split("```python")[1].split("```")[0].strip()
# else:
#         generated_code = generated_text
# # Print the generated text
# print(f"Instruction:{instruction}")
# print(f"Ground truth output:\n{gt}")
# print(f"Generated Answer:\n{generated_code}")


In [35]:
def generate_response(model, tokenizer, query, context=None):
    # Ensure that the model and input tensors are on the same device
    device = next(model.parameters()).device
    
    # Tokenize inputs
    inputs = tokenizer(query, context, return_tensors="pt", padding=True, add_special_tokens=True)
    input_ids = inputs["input_ids"].to(device)
    
    # Generate output
    output = model.generate(input_ids=input_ids,max_new_tokens =256, temperature = 0.3,top_k=1,num_beams=1)
    #output = model.generate(input_ids=input_ids,max_new_tokens =256)
    
    # Decode and return response
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response


In [22]:
eval_dataset = eval_dataset["test"]

In [36]:
# Initialize lists to store results
pred = []
iter = 0
gt =[]
# Iterate through the queries
number_of_eval_samples = 50
dataseteval = eval_dataset.select(range(number_of_eval_samples))
for query in dataseteval:  
    instruction = query["instruction"]
    ground = query["output"]
    gt.append(query["output"])
        
    # Define your prompt using the instruction variable
    prompt = f"""<start_of_turn>{instruction}<end_of_turn>
    <start_of_turn>model"""
    input_prompt = f"<s>[INST] {prompt} [/INST]"

    # Generate response without RAG
    input_prompt = f"{input_prompt}"
    respons = generate_response(model, tokenizer, input_prompt)
    #print(respons)
    generated_text = respons
        #print(result[0]['generated_text'])
    if "```python" in generated_text and "```" in generated_text:
        generated_code = generated_text.split("```python")[1].split("```")[0].strip()
    else:
        generated_code = generated_text
        
    marker = '<start_of_turn>model [/INST]'

    if marker in generated_text:
        index = generated_text.index(marker)
        generated_code = generated_text[index + len(marker):].strip()
    else:
        generated_code = generated_text
        
    pred.append(generated_code)

    # Print the generated text
    print(f"Instruction:{instruction}")
    print(f"Ground truth output:\n{ground}")    
    print(f"Generated Answer:\n{generated_code}")
    print(f"sample:{iter}")
    iter+=1

/home/qvr905/.conda/envs/mis/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.3` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/qvr905/.conda/envs/mis/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:509: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


Instruction:Create a Python program that encodes a given passage using the Caesar Cipher algorithm plaintext = "This is a secret message."
caesar_key = 0
Ground truth output:
```python
def caesar_cipher(plaintext, caesar_key):
    ciphertext = ""

    for char in plaintext:
        if char.isalpha():
            ascii_char = ord(char) - caesar_key
            if ascii_char < 97:
                ascii_char += 26
            ciphertext += chr(ascii_char)
        else:
            ciphertext += char

    return ciphertext

plaintext = "This is a secret message."
caesar_key = 0

encoded_message = caesar_cipher(plaintext, caesar_key)
print(encoded_message)
```
Generated Answer:
Sure, here is a Python program that encodes a given passage using the Caesar Cipher algorithm with a given key:

```python
import string

def caesar_encode(plaintext, key):
    plaintext = plaintext.lower()
    plaintext = plaintext.replace(string.punctuation, '')
    plaintext = plaintext.replace(' ', '')
    plaint

In [25]:
from tqdm import tqdm
import evaluate
from codebleu import calc_codebleu
from bert_score import score as bert_score

# Load evaluation metrics
bleu_metric = evaluate.load("bleu")
rouge_metric = evaluate.load("rouge")
bert_metric = evaluate.load("bertscore")
codebleu_metric = evaluate.load("k4black/codebleu")

def evaluate(predictions, references,model):    
    # Compute BERTScore
    #bert_score_precision, bert_score_recall, bert_score_f1= bert_metric.compute(predictions=predictions, references=references, lang='en', verbose=False)
    bert_score_precision, bert_score_recall, bert_score_f1 = bert_score(predictions, references, lang="en", verbose=False)
    #bert_score = bert_score_result['bertscore'].item()
    # Compute BLEU
    bleu_score = bleu_metric.compute(predictions=predictions, references=references,smooth=True)['bleu']
    
    # Compute Rouge-L
    rouge_score = rouge_metric.compute(predictions=predictions, references=references)['rougeL']
    
    codebleu = calc_codebleu(references, predictions, lang="python", weights=(0.25, 0.25, 0.25, 0.25), tokenizer=None)
    return bleu_score, rouge_score,codebleu, bert_score_precision, bert_score_recall, bert_score_f1


In [37]:
# Evaluate metrics
bleu, rouge ,codebleu, bert_score_precision, bert_score_recall, bert_score_f1= evaluate(pred, gt,model)
print(str(bleu))
print(str(rouge))
print(str(codebleu))
print(bert_score_precision.mean().item(), bert_score_recall.mean().item(), bert_score_f1.mean().item())



Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


0.19188503626135484
0.24070963675323875
{'codebleu': 0.33135745002035333, 'ngram_match_score': 0.052699095506998694, 'weighted_ngram_match_score': 0.1312102367381571, 'syntax_match_score': 0.5526315789473685, 'dataflow_match_score': 0.5888888888888889}
0.8466113209724426 0.8874513506889343 0.8662005066871643


: 

References: 
https://www.philschmid.de/fine-tune-llms-in-2024-with-trl